<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/Initial_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the victim's gender using an evolutionary algorithm to select features.

## - Initial Population

This code has all steps for the initial population. Where:

* Imports: the necessary libraries
* Scripts: imports the encoded dataset and create the initial population with 100 individuals, where each individual has 18 features, the first is the model and all others correspond for the features for the dataset
* Setups: results and export.

##Imports

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from pandas import read_csv 

## Scripts

In [2]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [3]:
def population(ind_size,pop_size,models):
  ind = []
  pop = []

  for p in range(pop_size):
    ind = [random.choice([True,False]) for i in range(ind_size)]
    ind[0] = random.choice(range(0,len(models)+1))
    pop.append(ind)
    ind = []

  print("-------------------------- Initial Population --------------------------")
  print(pd.DataFrame(pop, columns=['model', 'feature 1', 'feature 2', 'feature 3','feature 4',
                                   'feature 5', 'feature 6', 'feature 7','feature 8', 'feature 9',
                                   'feature 10', 'feature 11','feature 12', 'feature 13', 'feature 14',
                                   'feature 15','feature 16', 'feature 17']))
  return pop

## Instances

In [8]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_per_encoded')

In [9]:
models = [DecisionTreeClassifier(max_depth=10),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=14),
          RandomForestClassifier(max_depth=10),RandomForestClassifier(max_depth=14)]

In [10]:
initial_population = population(18,100,models)

-------------------------- Initial Population --------------------------
    model  feature 1  feature 2  ...  feature 15  feature 16  feature 17
0       3      False      False  ...       False        True        True
1       0      False      False  ...       False        True       False
2       0       True       True  ...        True        True        True
3       1       True       True  ...        True        True        True
4       0       True      False  ...        True        True       False
..    ...        ...        ...  ...         ...         ...         ...
95      3       True      False  ...        True       False        True
96      0       True      False  ...        True       False       False
97      1       True      False  ...       False        True        True
98      1       True      False  ...        True        True       False
99      3       True      False  ...       False       False       False

[100 rows x 18 columns]


In [7]:
#export population
with open("/content/drive/My Drive/Data Files/initial_population.txt", "wb") as fp: 
  pickle.dump(initial_population, fp)